# Copyright

<PRE>
Jelen iPython notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott "Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült. 
A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A notebook bármely részének újra felhasználása, publikálása csak a szerzők írásos beleegyezése esetén megegengedett.

2017 (c) Gyires-Tóth Bálint (toth.b kukac tmit pont bme pont hu)
</PRE>

# Regresszió mély tanulással

Ebben a notebookban a gépi tanulás egyik "Hello world"-jét fogjuk megismeri. Ehhez a Boston Housing Prices adatbázist fogjuk felhasználni. 

Az adatbázis formátumának leírását itt találjuk: https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.names
Magát az adatbázist pedig innen tölthetjük le: https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data

Töltsük le az utóbbi CSV fájlt és mentsük a jelenlegi Python forrás / notebook mellé.

Következő lépésként importáljuk a szükséges modulokat és állítsunk be egy random seed-et:

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import copy
import pandas as pd
from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler


np.random.seed(123)

Using TensorFlow backend.


Ezután töltsük le és töltsük be az adatokat, válasszuk szét a train-validation-test adathalmazokat és standardizáljuk a bemeneteket:

In [2]:
import urllib.request
urllib.request.urlretrieve ("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data", "housing.data")

('housing.data', <http.client.HTTPMessage at 0x7fa32f337d68>)

In [3]:
df = pd.read_csv("housing.data", delim_whitespace=True, header=None)
dataset = df.values
test_split = 0.1
valid_split = 0.2

X = dataset[:,0:13]
Y = dataset[:,13]

v_length = int(X.shape[0]*(1-valid_split-test_split))
t_length = int(X.shape[0]*(1-test_split))

X_test = X[t_length:]
Y_test = Y[t_length:]
X_valid = X[v_length:t_length]
Y_valid = Y[v_length:t_length]
X = X[:v_length]
Y = Y[:v_length]

scaler = StandardScaler().fit(X)
X = scaler.transform(X)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)


Ezt követően adjuk meg a callback függvényeket:

In [4]:
patience=20
early_stopping=EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=1)

És hozzuk létre a hálót és tanítsuk be:

In [5]:
model = Sequential()
model.add(Dense(output_dim=100, input_dim=X.shape[1]))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim=1, activation='linear'))

sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mse', optimizer=sgd)
history=model.fit(X,Y,epochs=1000, 
                  batch_size=10,
                  verbose=2,
                  validation_data=(X_valid, Y_valid),
                  callbacks=[checkpointer, early_stopping])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=13, units=100)`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="linear")`
  """


Train on 354 samples, validate on 101 samples
Epoch 1/1000
Epoch 00000: val_loss improved from inf to 2238.04288, saving model to weights.hdf5
1s - loss: 160.6470 - val_loss: 2238.0429
Epoch 2/1000
Epoch 00001: val_loss did not improve
0s - loss: 43.9657 - val_loss: 3807.4827
Epoch 3/1000
Epoch 00002: val_loss did not improve
0s - loss: 25.6695 - val_loss: 2289.2290
Epoch 4/1000
Epoch 00003: val_loss did not improve
0s - loss: 25.3248 - val_loss: 2914.9482
Epoch 5/1000
Epoch 00004: val_loss did not improve
0s - loss: 19.8369 - val_loss: 3750.2526
Epoch 6/1000
Epoch 00005: val_loss did not improve
0s - loss: 19.5895 - val_loss: 3093.3275
Epoch 7/1000
Epoch 00006: val_loss did not improve
0s - loss: 17.1458 - val_loss: 3032.7960
Epoch 8/1000
Epoch 00007: val_loss did not improve
0s - loss: 21.1266 - val_loss: 4971.2608
Epoch 9/1000
Epoch 00008: val_loss did not improve
0s - loss: 18.2996 - val_loss: 4268.2685
Epoch 10/1000
Epoch 00009: val_loss did not improve
0s - loss: 19.5739 - val_lo

A tanítás végeztével visszatöltjük a legjobb modellt, és kiértékeljük ennek teljesítményét a teszt adatokon. Mit jelenthet ez az érték?

In [7]:
model = load_model('weights.hdf5')

test_err = model.evaluate(X_test, Y_test)
print("\nTeszt hiba: %f" % (test_err))

32/51 [=================>............] - ETA: 0s
Teszt hiba: 71.303842
